<a href="https://colab.research.google.com/github/izmare/Flux_Dev_i2i/blob/main/FluxDevi2i.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Setup and Installation
#@markdown This cell clones the repository, sets up the environment, and installs necessary dependencies.

# Clone the repository and set the working directory
%cd /content
!git clone -b totoro3 https://github.com/camenduru/ComfyUI /content/TotoroUI
%cd /content/TotoroUI

# Install dependencies
!pip install -q torchsde einops diffusers accelerate xformers==0.0.27.post2
!apt -y install -qq aria2

# Create directories for models
import os
unet_dir = "/content/TotoroUI/models/unet/"
vae_dir = "/content/TotoroUI/models/vae/"
clip_dir = "/content/TotoroUI/models/clip/"
os.makedirs(unet_dir, exist_ok=True)
os.makedirs(vae_dir, exist_ok=True)
os.makedirs(clip_dir, exist_ok=True)

# Download model files
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8.safetensors -d /content/TotoroUI/models/unet -o flux1-dev-fp8.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft -d /content/TotoroUI/models/vae -o ae.sft
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors -d /content/TotoroUI/models/clip -o clip_l.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp8_e4m3fn.safetensors -d /content/TotoroUI/models/clip -o t5xxl_fp8_e4m3fn.safetensors

print("Setup completed.")


In [4]:
#@title Model Setup and Installation
#@markdown This cell sets up the environment and downloads selected models based on the checkboxes.

import os
import urllib.request
from tqdm.notebook import tqdm

# Directories for model storage
unet_dir = "/content/TotoroUI/models/unet/"
vae_dir = "/content/TotoroUI/models/vae/"
clip_dir = "/content/TotoroUI/models/clip/"

# URLs for model files
urls = {
    "flux_dev": {
        "unet": "https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8.safetensors",
        "vae": "https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft",
        "clip": [
            "https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors",
            "https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp8_e4m3fn.safetensors"
        ]
    },
    "flux_schnell": {
        "unet": "https://huggingface.co/camenduru/FLUX.1-schnell/resolve/main/flux1-schnell-fp8.safetensors",
        "vae": "https://huggingface.co/camenduru/FLUX.1-schnell/resolve/main/ae.sft",
        "clip": [
            "https://huggingface.co/camenduru/FLUX.1-schnell/resolve/main/clip_l.safetensors",
            "https://huggingface.co/camenduru/FLUX.1-schnell/resolve/main/t5xxl_fp8_e4m3fn.safetensors"
        ]
    }
}

# Checkbox options for model downloads
download_flux_dev = True  #@param {type:"boolean"}
download_flux_schnell = False  #@param {type:"boolean"}

# Function to download files with a progress bar
def download_file(url, output_path):
    if not os.path.exists(output_path):
        with tqdm(total=100, desc=f"Downloading {os.path.basename(output_path)}", unit="B", unit_scale=True) as pbar:
            def reporthook(block_num, block_size, total_size):
                pbar.update(block_num * block_size / total_size * 100)
            urllib.request.urlretrieve(url, output_path, reporthook=reporthook)

# Download selected model files
if download_flux_dev:
    if not os.path.exists(os.path.join(unet_dir, "flux1-dev-fp8.safetensors")):
        download_file(urls["flux_dev"]["unet"], os.path.join(unet_dir, "flux1-dev-fp8.safetensors"))
    if not os.path.exists(os.path.join(vae_dir, "ae.sft")):
        download_file(urls["flux_dev"]["vae"], os.path.join(vae_dir, "ae.sft"))
    for clip_url in urls["flux_dev"]["clip"]:
        if not os.path.exists(os.path.join(clip_dir, os.path.basename(clip_url))):
            download_file(clip_url, os.path.join(clip_dir, os.path.basename(clip_url)))

if download_flux_schnell:
    if not os.path.exists(os.path.join(unet_dir, "flux1-schnell-fp8.safetensors")):
        download_file(urls["flux_schnell"]["unet"], os.path.join(unet_dir, "flux1-schnell-fp8.safetensors"))
    if not os.path.exists(os.path.join(vae_dir, "ae.sft")):
        download_file(urls["flux_schnell"]["vae"], os.path.join(vae_dir, "ae.sft"))
    for clip_url in urls["flux_schnell"]["clip"]:
        if not os.path.exists(os.path.join(clip_dir, os.path.basename(clip_url))):
            download_file(clip_url, os.path.join(clip_dir, os.path.basename(clip_url)))

print("Model setup completed.")


Model setup completed.


In [5]:
#@title Anime Style Image Generation
#@markdown This code generates an image in an anime style using the selected model. The output file name is incrementally generated to prevent overwriting.

import random
import os
from PIL import Image
import numpy as np
import torch
import nodes
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro_extras import nodes_post_processing
from totoro import model_management

# Import modules from the repository
import sys
sys.path.append('/content/TotoroUI')

# Load classes from nodes
DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
VAEEncode = NODE_CLASS_MAPPINGS["VAEEncode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()
ImageScaleToTotalPixels = nodes_post_processing.NODE_CLASS_MAPPINGS["ImageScaleToTotalPixels"]()

# Generate an incremental file name
def generate_incremental_filename(base_dir="/content/", base_name="image", extension=".png"):
    i = 0
    while os.path.exists(os.path.join(base_dir, f"{base_name}_{i:04d}{extension}")):
        i += 1
    return os.path.join(base_dir, f"{base_name}_{i:04d}{extension}")

output_filename = generate_incremental_filename()

# Input fields
model_choice = "flux_dev"  #@param ["flux_dev", "flux_schnell"]
positive_prompt = "Pixel art anime girl, short white hair with short twintails, cat ears headband, red eyes, cheerful expression, black and red outfit, crop top, short skirt, belt with buckle, thigh-high striped stockings, boots with gold tips, dual flamethrower weapons, gold and black design, backpack fuel tank, 16-bit graphics, retro gaming aesthetic, limited color palette, chunky pixels, no anti-aliasing, sharp pixel edges, low-resolution sprite, SNES era style, 2D side-scrolling game character, simplified pixel details, blocky shapes, pixelated shading, classic arcade look, 8-bit inspired, old-school video game character, square pixels only, mosaic-like appearance, pixel-perfect edges, retro pixel art style, low color depth, nostalgic graphics, dithering effects, dynamic action pose, contrast between light hair and dark outfit"  #@param {type:"string"}
width = 512  #@param {type:"slider", min:256, max:2048, step:64}
height = 1024  #@param {type:"slider", min:256, max:2048, step:64}
seed = 0  #@param {type:"number"}
steps = 20  #@param {type:"slider", min:10, max:100, step:5}
sampler_name = "euler"  #@param ["euler", "ddim", "plms", "heun"]
scheduler = "simple"  #@param ["simple", "linear", "cosine"]
guidance_scale = 7.5  #@param {type:"slider", min:1.0, max:20.0, step:0.5}
image_dir = "/content/IconRole24_webp_resized.png"  #@param {type:"string"}

# Convert input image to PNG if necessary
image_ext = os.path.splitext(image_dir)[-1].lower()
if image_ext not in [".png"]:
    img = Image.open(image_dir)
    image_dir = os.path.splitext(image_dir)[0] + ".png"
    img.save(image_dir)

# Load models based on user choice
with torch.inference_mode():
    if model_choice == "flux_dev":
        clip = DualCLIPLoader.load_clip("t5xxl_fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux")[0]
        unet = UNETLoader.load_unet("flux1-dev-fp8.safetensors", "fp8_e4m3fn")[0]
        vae = VAELoader.load_vae("ae.sft")[0]
    elif model_choice == "flux_schnell":
        clip = DualCLIPLoader.load_clip("t5xxl_fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux")[0]
        unet = UNETLoader.load_unet("flux1-schnell-fp8.safetensors", "fp8_e4m3fn")[0]
        vae = VAELoader.load_vae("ae.sft")[0]

    if seed == 0:
        seed = random.randint(0, 18446744073709551615)
    print(f"Seed: {seed}")

    cond, pooled = clip.encode_from_tokens(clip.tokenize(positive_prompt), return_pooled=True)
    cond = [[cond, {"pooled_output": pooled}]]
    noise = RandomNoise.get_noise(seed)[0]
    guider = BasicGuider.get_guider(unet, cond, guidance_scale=guidance_scale)[0]
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = BasicScheduler.get_sigmas(unet, scheduler, steps, 0.75)[0]

    image = nodes.LoadImage().load_image(image_dir)[0]
    latent_image = ImageScaleToTotalPixels.upscale(image, "lanczos", 1.0)[0]
    latent_image = VAEEncode.encode(vae, latent_image)[0]

    sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
    model_management.soft_empty_cache()
    decoded = VAEDecode.decode(vae, sample)[0].detach()
    Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save(output_filename)

Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0])


ModuleNotFoundError: No module named 'totoro_extras'